In [3]:
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd

In [93]:
def angular_dist_score(az_true, zen_true, az_pred, zen_pred):
    '''
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse 
    cosine (arccos) thereof is then the angle between the two input vectors
    
    Parameters:
    -----------
    
    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian
    
    Returns:
    --------
    
    dist : float
        mean over the angular distance(s) in radian
    '''
    
    if not (np.all(np.isfinite(az_true)) and
            np.all(np.isfinite(zen_true)) and
            np.all(np.isfinite(az_pred)) and
            np.all(np.isfinite(zen_pred))):
        raise ValueError("All arguments must be finite")
    
    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)
    
    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)
    
    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod =  np.clip(scalar_prod, -1, 1)
    
    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

In [4]:
# Importing data
batch1 = pd.read_parquet('../batches_train/batch_1_repartitions/part.0.parquet')
sensor_geom = pd.read_csv('../sensor_geometry.csv')

In [5]:
# Importing meta data
meta = pd.read_parquet('../batches_train/train_meta.parquet')

In [7]:
event_ids = list(set(batch1.index))

In [14]:
# Part of meta that we need
meta_batch1 = meta[meta.event_id.isin(event_ids)]
meta_batch1.set_index('event_id',
                      inplace=True)
meta_batch1.drop(['batch_id', 'first_pulse_index', 'last_pulse_index'], 
                 axis=1,
                 inplace=True)

In [21]:
meta_batch1.head()

,azimuth,zenith
event_id,,
24,5.029555,2.087498
41,0.417742,1.549686
59,1.160466,2.401942
67,5.845952,0.759054
72,0.653719,0.939117


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
train_indices, test_indices = train_test_split(event_ids,
                                               shuffle=True,
                                               random_state=123,
                                               test_size=0.25)

In [24]:
num_sensors = 5160

In [75]:
df_train = pd.DataFrame(columns=[i for i in range(0, num_sensors)]+['az','ze'])

In [76]:
# Now we generate feature data from the raw data
# We use aux = False only here
count = 0

batch1_no_aux = batch1[batch1.auxiliary==False]

for index in train_indices[:1000]:
    event = batch1_no_aux.loc[index]

    sensors = event.sensor_id.values

    for sensor in sensors:
        df_train.loc[index,sensor] = 1
    
    az, ze = meta_batch1.loc[index,['azimuth','zenith']].values
    df_train.loc[index,'az'] = az
    df_train.loc[index,'ze'] = ze

    count = count + 1
    if count % 100 == 0:
        print("Working on event", count)

Working on event 100
Working on event 200
Working on event 300
Working on event 400
Working on event 500
Working on event 600
Working on event 700
Working on event 800
Working on event 900
Working on event 1000


In [77]:
df_train.fillna(0, inplace=True)

In [78]:
# This is the DataFrame that represents our training data
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,5152,5153,5154,5155,5156,5157,5158,5159,az,ze
1023496,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2.248502,0.432784
681492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.299984,2.809545
38448,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5.291643,0.989836
892244,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.758019,2.730688
717306,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3.283212,0.080609


In [79]:
slr1 = LinearRegression(copy_X=True)

In [80]:
Xs_train = df_train[range(0,num_sensors)]
ys_train = df_train[['az','ze']]

In [81]:
slr1.fit(Xs_train.values, ys_train.values)

LinearRegression()

In [82]:
slr1.coef_.shape

(2, 5160)

In [83]:
# Now we set up the testing data
df_test = pd.DataFrame(columns=[i for i in range(0, num_sensors)]+['az','ze'])

count = 0

batch1_no_aux = batch1[batch1.auxiliary==False]

for index in test_indices[:300]:
    event = batch1_no_aux.loc[index]

    sensors = event.sensor_id.values

    for sensor in sensors:
        df_test.loc[index,sensor] = 1
    
    az, ze = meta_batch1.loc[index,['azimuth','zenith']].values
    df_test.loc[index,'az'] = az
    df_test.loc[index,'ze'] = ze

    count = count + 1
    if count % 100 == 0:
        print("Working on event", count)

df_test.fillna(0, inplace=True)

Working on event 100
Working on event 200
Working on event 300


In [84]:
Xs_test = df_test[range(0,num_sensors)]
ys_test = df_test[['az','ze']]

In [85]:
ys_test.values

array([[4.75980396, 1.40686725],
       [1.68099892, 1.13648942],
       [4.96291399, 2.05326109],
       [1.14900089, 1.80030995],
       [0.646665  , 1.73569752],
       [0.12511552, 2.53903303],
       [5.45066336, 2.73013905],
       [2.53985376, 0.83210598],
       [3.49800602, 0.83857345],
       [0.35263274, 2.15120563],
       [2.38950211, 1.70531374],
       [6.10459243, 1.3498337 ],
       [0.18348714, 0.35711182],
       [1.79278441, 2.00269534],
       [4.53283201, 0.48252099],
       [5.30524791, 0.53594885],
       [2.53250525, 0.92799401],
       [5.70195407, 2.32208362],
       [5.02460259, 1.89096533],
       [4.45316056, 1.71181461],
       [0.24218569, 1.50965491],
       [4.94586755, 1.38187845],
       [0.19974   , 0.80581423],
       [2.01324259, 0.21537926],
       [2.9788118 , 2.31456226],
       [5.44943646, 2.86957857],
       [4.44429704, 1.2468127 ],
       [3.84606783, 2.17405789],
       [3.93946867, 1.70710141],
       [2.32321878, 2.08035257],
       [3.

In [101]:
slr1.predict(Xs_test.values).shape

(300, 2)

### Baseline model

This model just "trains" by computing the mean of the output azimuth and zenith and predicts these numbers always

In [89]:
train_mean = np.mean(ys_train.values, axis=0)

In [90]:
train_mean

array([3.33623343, 1.52299244])

In [95]:
baseline_pred_az = train_mean[0]*np.ones(len(ys_test))
baseline_pred_ze = train_mean[1]*np.ones(len(ys_test))

In [100]:
baseline_error = angular_dist_score(ys_test['az'].values,
                                    ys_test['ze'].values,
                                    baseline_pred_az,
                                    baseline_pred_ze)

print("The angular distance score for the baseline model is", baseline_error)

The angular distance score for the baseline model is 1.5288671288320204


So the baseline model is accurate up to basically $\pi/2$ radians, i.e. statistically speaking the true direction will be in the same hemisphere that the baseline model guesses.

### Model 1

This model just puts a 1 in a sensor where it went off for that event, and a 0 in the sensors that didn't go off

In [103]:
az_pred_1 = slr1.predict(Xs_test.values)[:,0]
ze_pred_1 = slr1.predict(Xs_test.values)[:,1]

model_1_error = angular_dist_score(ys_test['az'].values,
                                   ys_test['ze'].values,
                                   az_pred_1,
                                   ze_pred_1)

print("The angular distance score for model 1 is", model_1_error)

The angular distance score for model 1 is 1.5847234132004213
